首先，构造一个super triangle，并完成可视化
 - https://tutorial.pyvista.org/tutorial/01_basic/index.html
    



In [6]:
import numpy as np
import pyvista as pv
from typing import List

def get_N_points(N=50, dim=2) -> np.array:
    np.random.seed(0)
    points = np.random.rand(N, dim)
    if dim==2:
        x = np.zeros((N,1))
        points = np.hstack((points, x))
    return points

def gen_Voronoi(vert):
    pass

def within_circle(a, b, c, d) -> bool:
    '''
    return if d is inside circle composed of a, b, c
    '''
    det = np.array(
        [
            [a[0]-d[0], a[1]-d[1], (a[0]-d[0])**2+(a[1]-d[1])**2],
            [b[0]-d[0], b[1]-d[1], (b[0]-d[0])**2+(b[1]-d[1])**2],
            [c[0]-d[0], c[1]-d[1], (c[0]-d[0])**2+(c[1]-d[1])**2]
        ]
    )
    det = np.linalg.det(det)
    if det > 0:
        return True
    return False

def get_AABB(points, offset=0.1) -> List:
    '''
    return coordination of bottom_left(with offset), delta x and delta y
    '''
    points = np.array(points)
    top_right = (max(points[:,0]) + offset, max(points[:,1]) + offset)
    bottom_left = (min(points[:,0]) - offset, min(points[:,1]) - offset)
    return [bottom_left[0] - offset, 
            bottom_left[1] - offset, 
            top_right[0] - bottom_left[0] + 2*offset,
            top_right[1] - bottom_left[1] + 2*offset,
            ]

def get_super_triangle(points) -> List:
    '''
    get a triangle within all points
    通过AABB的外接圆构造外接正三角形得到
           v1
        //    \\
      v2   ==  v3      
    return [
        [x1, y1, 0], #v1
        [x2, y2, 0], #v2
        [x3, y3, 0], #v3
    ]
    '''
    p_x, p_y, delta_x, delta_y = get_AABB(points)
    center = (p_x+0.5*delta_x, p_y+0.5*delta_y)
    r = ((0.5*delta_x)**2+(0.5*delta_y)**2)**0.5
    v1 = [center[0], center[1] + 2*r, 0]
    v2 = [center[0] - 3**0.5*r, center[1] - r, 0]
    v3 = [center[0] + 3**0.5*r, center[1] - r, 0]
    return [v1, v2, v3]

def show_super_triangle(show_AABB=False):
    p = pv.Plotter(off_screen=True)
    points = get_N_points()
    # mesh = pv.PolyData(points, )
    
    v1, v2, v3 = get_super_triangle(points)

    p.add_points(points, color='red', point_size=20)
    p.add_lines(np.array([v1, v2, v2, v3, v3, v1]), color='blue', connected=True)
    if show_AABB:
        p_x, p_y, delta_x, delta_y = get_AABB(points)
        p.add_lines(np.array([[p_x, p_y, 0], [p_x + delta_x, p_y, 0], 
                            [p_x, p_y, 0], [p_x , p_y + delta_y, 0],
                            [p_x, p_y + delta_y, 0], [p_x + delta_x, p_y + delta_y, 0],                 
                            [p_x + delta_x, p_y, 0], [p_x + delta_x, p_y + delta_y, 0]           
                            ]), color='green', connected=True)        
    p.show()

In [7]:
show_super_triangle(show_AABB=True)

Widget(value="<iframe src='http://localhost:64017/index.html?ui=P_0x285e4b4f520_1&reconnect=auto' style='width…

在这里，我们实现[Watson_algorithm](https://en.wikipedia.org/wiki/Bowyer%E2%80%93Watson_algorithm)，通过以下步骤

- 创建三个点，使得构造出的三角形（super-triangle）包含所有已存在的点
- 置一个triangulation的空list，将super-triangle添加到triangulation的list中
- 遍历每个点，对于每个点$p$：
  - 设置一个badTriangles的空list
  - 遍历triangulation的list，如果存在list中的三角形，使得$p$在三角形的外接圆内，则将该三角形添加到badTriangles的list中去（这里我们使用[行列式](https://en.wikipedia.org/wiki/Delaunay_triangulation#Algorithms)计算，注意使用这个算法顶点要**保证是counterwise**的）
  - 构造点$p$对应的polygonal hole：设置一个polygon的空list，遍历badTriangles的每个三角形，对于每个三角形$A$，遍历它的每一条边，如果这条边不被badTriangles中的其他三角形所共享，则将这条边添加到polygon中
  - 将triangulation中的badTriangles删除
  - 对于点$p$和polygon中的每一条边，可以构造出一个三角形；将所有这些三角形添加到triangulation中

- 过河拆桥，将triangulation中所有包含super-triangle顶点的三角形删除



In [3]:

class Edge:
    def __init__(self, v1, v2, dim=2):
        self.dim = dim
        for i in range(dim): # ensure v1 < v2
            if v1[i] > v2[i]:
                self.v1, self.v2 = v2, v1
                break
            if v1[i] < v2[i]:
                self.v1, self.v2 = v1, v2
                break
        else:
            print("Error: same point can't compose an edge")

    def __eq__(self, obj):
        for i in range(self.dim):
            if self.v1[i]!=obj.v1[i] or self.v2[i]!=obj.v2[i]:
                return False
        return True


class Triangle:
    def __init__(self, v, dim=2):
        self.dim = dim
        self.load_counterwise(v)
        self.points = [self.v1, self.v2, self.v3]
        self.edges = [Edge(self.v1, self.v2), Edge(self.v2, self.v3), Edge(self.v3, self.v1)]

    def load_counterwise(self, v):
        A, B, C = np.array(v[0]), np.array(v[1]), np.array(v[2])
        AB = B - A
        BC = C - B
        
        cross_product = np.cross(AB, BC)

        if np.any(cross_product > 0):
            self.v1, self.v2, self.v3 = v[0], v[1], v[2]
        elif np.any(cross_product < 0):
            self.v1, self.v2, self.v3 = v[0], v[2], v[1]

    def within_circle(self, point) -> bool:
        '''
        return if point is inside circle 
        '''
        a, b, c, d = self.v1, self.v2, self.v3, point
        det = np.array(
            [
                [a[0]-d[0], a[1]-d[1], (a[0]-d[0])**2+(a[1]-d[1])**2],
                [b[0]-d[0], b[1]-d[1], (b[0]-d[0])**2+(b[1]-d[1])**2],
                [c[0]-d[0], c[1]-d[1], (c[0]-d[0])**2+(c[1]-d[1])**2]
            ]
        )
        det = np.linalg.det(det)
        if det > 0:
            return True
        return False

    def has_point(self, point):
        for v in self.points:
            cnt = 0
            for i in range(self.dim):
                if point[i] == v[i]:
                    cnt += 1
            if cnt == self.dim:
                return True
        return False


def edge_in_tris(edge, tris):
    cnt = 0
    for tri in tris:
        for e in tri.edges:
            if edge == e:
                cnt += 1
    return cnt

def BowyerWatson(points: np.array, step=None):
    triangulation = [] # step.1
    super_triangle = Triangle(get_super_triangle(points))
    triangulation.append(super_triangle) # step.2

    for i in range(points.shape[0]): # step.3
        badTriangles = [] # step.3.1
        p = points[i]
        # print("[ stage ", i, "]: length of triangulation: ", len(triangulation), "length of badTriangles: ", len(badTriangles))
        for tri in triangulation:
            if tri.within_circle(p)==True:
                badTriangles.append(tri) # step.3.2
        # print("[ stage ", i, "]: length of triangulation: ", len(triangulation), "length of badTriangles: ", len(badTriangles))
        polygon = [] # step.3.3
        for tri in badTriangles:
            for edge in tri.edges:
                if edge_in_tris(edge, badTriangles)==1: # not shared by other bad-triangle
                    polygon.append(edge)
        # print("[ stage ", i, "]: length of polygon: ", len(polygon))
        # print(polygon)
        triangulation = list(set(triangulation) - set(badTriangles)) # step.3.4
        for edge in polygon:
            triangulation.append(Triangle([p, edge.v1, edge.v2])) # step3.5
        # print("[ stage ", i, "]: length of triangulation: ", len(triangulation), "length of badTriangles: ", len(badTriangles))
        if step!= None and i==step:
            return polygon, triangulation # for visualization of mid-steps
    del_tri = set()
    for p in super_triangle.points: # step.4
        for tri in triangulation:
            if tri.has_point(p):
                del_tri.add(tri)
    triangulation = list(set(triangulation)-del_tri)
    return triangulation


def show_triangulation(show_AABB=False, step=None):
    p = pv.Plotter()
    points = get_N_points()
    # mesh = pv.PolyData(points, )

    v1, v2, v3 = get_super_triangle(points)
    p.add_points(points, color='red', point_size=20)
    p.add_lines(np.array([v1, v2, v2, v3, v3, v1]), color='blue', connected=True)
    if show_AABB:
        p_x, p_y, delta_x, delta_y = get_AABB(points)
        p.add_lines(np.array([[p_x, p_y, 0], [p_x + delta_x, p_y, 0], 
                            [p_x, p_y, 0], [p_x , p_y + delta_y, 0],
                            [p_x, p_y + delta_y, 0], [p_x + delta_x, p_y + delta_y, 0],                 
                            [p_x + delta_x, p_y, 0], [p_x + delta_x, p_y + delta_y, 0]           
                            ]), color='green', connected=True)        
    
    if step==None:
        triangulation = BowyerWatson(points)
    else:
        polygon, triangulation = BowyerWatson(points, step=step) 

    for tri in triangulation:
        for e in tri.edges:
            p.add_lines(np.array([e.v1, e.v2]), color='black', connected=True)
    if step!=None:
        for e in polygon:
            p.add_lines(np.array([e.v1, e.v2]), color='yellow', connected=True)
        
    p.show() 

def get_triangluation_gif(show_AABB=False):
    p = pv.Plotter(notebook=False, off_screen=True)
    points = get_N_points()

    v1, v2, v3 = get_super_triangle(points)
    p.add_points(points, color='red', point_size=20)
    p.add_lines(np.array([v1, v2, v2, v3, v3, v1]), color='blue', connected=True)
    if show_AABB:
        p_x, p_y, delta_x, delta_y = get_AABB(points)
        p.add_lines(np.array([[p_x, p_y, 0], [p_x + delta_x, p_y, 0], 
                            [p_x, p_y, 0], [p_x , p_y + delta_y, 0],
                            [p_x, p_y + delta_y, 0], [p_x + delta_x, p_y + delta_y, 0],                 
                            [p_x + delta_x, p_y, 0], [p_x + delta_x, p_y + delta_y, 0]           
                            ]), color='green', connected=True) 
    
    p.open_gif("../triangluation.gif")
    p.write_frame()
    for frame in range(points.shape[0]):
        polygon, triangulation = BowyerWatson(points, step=frame)

        # ------clear and reload------
        p.clear()
        p.add_points(points, color='red', point_size=20)
        p.add_lines(np.array([v1, v2, v2, v3, v3, v1]), color='blue', connected=True)
        if show_AABB:
            p_x, p_y, delta_x, delta_y = get_AABB(points)
            p.add_lines(np.array([[p_x, p_y, 0], [p_x + delta_x, p_y, 0], 
                                [p_x, p_y, 0], [p_x , p_y + delta_y, 0],
                                [p_x, p_y + delta_y, 0], [p_x + delta_x, p_y + delta_y, 0],                 
                                [p_x + delta_x, p_y, 0], [p_x + delta_x, p_y + delta_y, 0]           
                                ]), color='green', connected=True) 
        # ------clear and reload------

        for tri in triangulation:
            for e in tri.edges:
                p.add_lines(np.array([e.v1, e.v2]), color='black', connected=True)
        for e in polygon:
            p.add_lines(np.array([e.v1, e.v2]), color='yellow', connected=True)
        p.write_frame()

    # ------clear and reload------
    p.clear()
    p.add_points(points, color='red', point_size=20)
    p.add_lines(np.array([v1, v2, v2, v3, v3, v1]), color='blue', connected=True)
    if show_AABB:
        p_x, p_y, delta_x, delta_y = get_AABB(points)
        p.add_lines(np.array([[p_x, p_y, 0], [p_x + delta_x, p_y, 0], 
                            [p_x, p_y, 0], [p_x , p_y + delta_y, 0],
                            [p_x, p_y + delta_y, 0], [p_x + delta_x, p_y + delta_y, 0],                 
                            [p_x + delta_x, p_y, 0], [p_x + delta_x, p_y + delta_y, 0]           
                            ]), color='green', connected=True) 
    # ------clear and reload------
    triangulation = BowyerWatson(points)
    for tri in triangulation:
        for e in tri.edges:
            p.add_lines(np.array([e.v1, e.v2]), color='black', connected=True)
    p.write_frame()
    p.close()




    


        


    

In [5]:
# show_triangulation(show_AABB=False, step=5) # miss step gets final result
get_triangluation_gif(show_AABB=False)